In [556]:
import pandas as pd
import numpy as np
import os
from utilities.utils import SectoralDiffReport, ErrorFunctions, SSPModelForCalibration
from utilities.diff_reports import DiffReportUtils

In [557]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [558]:
# Define paths
curr_dir = os.getcwd()
misc_dir = os.path.join(curr_dir, 'misc')
sectoral_report_dir = os.path.join(misc_dir, 'sectoral_reports')
dummy_files_dir = os.path.join(misc_dir, 'dummy')
sectoral_report_mapping_dir = os.path.join(misc_dir, 'sectoral_report_mapping')


## Explore the new emissions targets csv

In [559]:
# Read emission targets csv
emission_targets = pd.read_csv(os.path.join(sectoral_report_mapping_dir, '2022_emission_targets.csv'))
emission_targets.head()


,Subsector,Gas,Edgar_Class,Edgar_Subsector,Edgar_Sector,Vars,ids,IND,PRY,KAZ,...,SLE,BEN,ETH,MRT,NER,TGO,IRN,HRV,GEO,Year
0,lvst,ch4,AG - Livestock:CH4,AG - Livestock,Agriculture,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,1:lvst:ch4,216.832747,12.036521,13.135347,...,0.517069,1.488482,45.031510,3.446201,12.146617,0.787989,9.529176,0.805474,0.996739,2022
1,lsmm,ch4,AG - Livestock:CH4,AG - Livestock,Agriculture,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,2:lsmm:ch4,216.832747,12.036521,13.135347,...,0.517069,1.488482,45.031510,3.446201,12.146617,0.787989,9.529176,0.805474,0.996739,2022
2,lsmm,n2o,AG - Livestock:N2O,AG - Livestock,Agriculture,emission_co2e_n2o_lsmm_anaerobic_digester:emis...,3:lsmm:n2o,2.169403,0.125988,1.269783,...,0.032553,0.061310,0.280803,0.010149,0.084756,0.100270,0.210769,0.100881,0.088307,2022
3,agrc,co2,AG - Crops:CO2,AG - Crops,Agriculture,emission_co2e_co2_agrc_biomass_bevs_and_spices...,4:agrc:co2,32.634819,0.076804,0.454098,...,0.000000,0.000000,0.465140,0.000000,0.000000,0.000000,1.858300,0.106978,0.100935,2022
4,agrc,ch4,AG - Crops:CH4,AG - Crops,Agriculture,emission_co2e_ch4_agrc_anaerobicdom_rice:emiss...,5:agrc:ch4,127.221401,1.101916,0.833077,...,1.327563,0.527888,0.645650,0.264313,0.309672,0.389620,3.137081,0.000115,0.003060,2022


In [560]:
# Cols to keep
cols_to_keep = ['Subsector', 'Gas', 'Edgar_Class', 'Edgar_Subsector', 'Edgar_Sector', 'Vars', 'HRV']
# Filter df
emission_targets = emission_targets[cols_to_keep]

In [561]:
# Edgar_Class is repeated for some rows
emission_targets.sort_values(by='Edgar_Class').head(20)

,Subsector,Gas,Edgar_Class,Edgar_Subsector,Edgar_Sector,Vars,HRV
4,agrc,ch4,AG - Crops:CH4,AG - Crops,Agriculture,emission_co2e_ch4_agrc_anaerobicdom_rice:emiss...,0.000115
3,agrc,co2,AG - Crops:CO2,AG - Crops,Agriculture,emission_co2e_co2_agrc_biomass_bevs_and_spices...,0.106978
5,agrc,n2o,AG - Crops:N2O,AG - Crops,Agriculture,emission_co2e_n2o_agrc_biomass_burning:emissio...,1.300449
0,lvst,ch4,AG - Livestock:CH4,AG - Livestock,Agriculture,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,0.805474
1,lsmm,ch4,AG - Livestock:CH4,AG - Livestock,Agriculture,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,0.805474
2,lsmm,n2o,AG - Livestock:N2O,AG - Livestock,Agriculture,emission_co2e_n2o_lsmm_anaerobic_digester:emis...,0.100881
65,ccsq,ch4,CCSQ:CH4,CCSQ,CCSQ,emission_co2e_ch4_ccsq_direct_air_capture,0.000000
66,ccsq,co2,CCSQ:CO2,CCSQ,CCSQ,emission_co2e_co2_ccsq_direct_air_capture,0.000000
67,ccsq,n2o,CCSQ:N2O,CCSQ,CCSQ,emission_co2e_n2o_ccsq_direct_air_capture,0.000000
7,scoe,ch4,EN - Building:CH4,EN - Building,Energy,emission_co2e_ch4_scoe_commercial_municipal:em...,0.332792


In [562]:
# Check the highest emissions
emission_targets.sort_values(by='HRV', ascending=False).head(20)

,Subsector,Gas,Edgar_Class,Edgar_Subsector,Edgar_Sector,Vars,HRV
18,trns,co2,EN - Transportation:CO2,EN - Transportation,Energy,emission_co2e_co2_trns_aviation:emission_co2e_...,6.654934
6,scoe,co2,EN - Building:CO2,EN - Building,Energy,emission_co2e_co2_scoe_commercial_municipal:em...,2.603821
9,entc,co2,EN - Electricity/Heat:CO2,EN - Electricity/Heat,Energy,emission_co2e_co2_entc_generation_pp_biogas:em...,2.522588
15,inen,co2,EN - Manufacturing/Construction:CO2,EN - Manufacturing/Construction,Energy,emission_co2e_co2_inen_agriculture_and_livesto...,2.370070
21,ippu,co2,IN - Industrial Processes:CO2,IN - Industrial Processes,Industrial Processes,emission_co2e_co2_ippu_product_use_product_use...,2.168521
5,agrc,n2o,AG - Crops:N2O,AG - Crops,Agriculture,emission_co2e_n2o_agrc_biomass_burning:emissio...,1.300449
54,waso,ch4,Waste - Solid Waste:CH4,Waste - Solid Waste,Waste,emission_co2e_ch4_waso_biogas_food:emission_co...,1.009457
1,lsmm,ch4,AG - Livestock:CH4,AG - Livestock,Agriculture,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,0.805474
0,lvst,ch4,AG - Livestock:CH4,AG - Livestock,Agriculture,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,0.805474
12,fgtv,co2,EN - Fugitive Emissions:CO2,EN - Fugitive Emissions,Energy,emission_co2e_co2_fgtv_fuel_coal:emission_co2e...,0.798061


In [563]:
emission_targets['weights'] = emission_targets['HRV'] / emission_targets['HRV'].sum()

In [564]:
emission_targets.sort_values(by='weights', ascending=False).head(20)

,Subsector,Gas,Edgar_Class,Edgar_Subsector,Edgar_Sector,Vars,HRV,weights
18,trns,co2,EN - Transportation:CO2,EN - Transportation,Energy,emission_co2e_co2_trns_aviation:emission_co2e_...,6.654934,0.357225
6,scoe,co2,EN - Building:CO2,EN - Building,Energy,emission_co2e_co2_scoe_commercial_municipal:em...,2.603821,0.139768
9,entc,co2,EN - Electricity/Heat:CO2,EN - Electricity/Heat,Energy,emission_co2e_co2_entc_generation_pp_biogas:em...,2.522588,0.135408
15,inen,co2,EN - Manufacturing/Construction:CO2,EN - Manufacturing/Construction,Energy,emission_co2e_co2_inen_agriculture_and_livesto...,2.370070,0.127221
21,ippu,co2,IN - Industrial Processes:CO2,IN - Industrial Processes,Industrial Processes,emission_co2e_co2_ippu_product_use_product_use...,2.168521,0.116402
5,agrc,n2o,AG - Crops:N2O,AG - Crops,Agriculture,emission_co2e_n2o_agrc_biomass_burning:emissio...,1.300449,0.069806
54,waso,ch4,Waste - Solid Waste:CH4,Waste - Solid Waste,Waste,emission_co2e_ch4_waso_biogas_food:emission_co...,1.009457,0.054186
1,lsmm,ch4,AG - Livestock:CH4,AG - Livestock,Agriculture,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,0.805474,0.043236
0,lvst,ch4,AG - Livestock:CH4,AG - Livestock,Agriculture,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,0.805474,0.043236
12,fgtv,co2,EN - Fugitive Emissions:CO2,EN - Fugitive Emissions,Energy,emission_co2e_co2_fgtv_fuel_coal:emission_co2e...,0.798061,0.042838


## Test and Sketch the new DiffReport Methods

In [565]:
dru = DiffReportUtils('HRV', os.path.join(sectoral_report_mapping_dir, 'edgar_ssp_cw.csv'), sectoral_report_dir, energy_model_flag=False)

In [566]:
edgar_emissions = dru.edgar_emission_db_etl(os.path.join(sectoral_report_mapping_dir, 'CSC-GHG_emissions-April2024_to_calibrate.csv'))
edgar_emissions

,iso_alpha_3,edgar_class,edgar_emission,year
0,HRV,AG - Livestock:CH4,1.682916,2015
1,HRV,AG - Livestock:N2O,0.110193,2015
2,HRV,AG - Crops:CH4,0.000227,2015
3,HRV,AG - Crops:CO2,0.095857,2015
4,HRV,AG - Crops:N2O,1.038055,2015
5,HRV,EN - Building:CH4,0.416708,2015
6,HRV,EN - Building:CO2,2.726867,2015
7,HRV,EN - Building:N2O,0.104122,2015
8,HRV,EN - Electricity/Heat:CH4,0.004396,2015
9,HRV,EN - Electricity/Heat:CO2,3.297328,2015


In [567]:
ssp_out = pd.read_csv(os.path.join(dummy_files_dir, 'ssp_output_no_energy_dummy.csv'))

In [568]:
ssp_out.head()

,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,area_agrc_crops_other_woody_perennial,area_agrc_crops_pulses,...,yield_agrc_fruits_tonne,yield_agrc_herbs_and_other_perennial_crops_tonne,yield_agrc_nuts_tonne,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne
0,0,128992.475401,286925.570284,59534.988647,155176.845408,200241.524314,109285.291660,144565.284932,96468.731604,66425.612333,...,860714.235864,5.781814e+06,65829.816853,279108.581486,50211.974800,175242.943363,0.0,0.0,2.050396e+06,725849.288265
1,1,128305.935425,285398.458926,59218.124042,154350.943684,199175.773688,108703.639735,143795.861390,95955.293587,66072.073584,...,857478.660027,5.906717e+06,52166.876709,331312.854435,70450.376552,215192.338859,2736.0,2449.0,2.713049e+06,699533.070417
2,2,127621.990578,283877.120067,58902.457190,153528.163879,198114.051612,108124.186462,143029.346278,95443.796372,65719.871216,...,660924.245615,5.951148e+06,38967.956801,296921.771405,82224.830575,210243.344677,2736.0,2449.0,2.331104e+06,761148.281366
3,3,126940.639041,282361.549661,58587.987250,152708.503803,197056.355263,107546.930298,142265.737557,94934.238599,65369.004292,...,957822.768981,5.919376e+06,17239.772927,378730.362142,93529.211867,213539.618942,2736.0,2449.0,2.111415e+06,729155.981592
4,4,126261.879236,280851.744198,58274.713494,151891.961560,196002.682190,106971.869908,141505.033460,94426.619087,65019.472000,...,892950.504956,5.887725e+06,21001.252360,359927.486275,85408.876964,264501.812874,2736.0,2449.0,2.234268e+06,737682.556359


In [569]:
print(ssp_out['time_period'].dtype)  # Check the current type
print(ssp_out['time_period'].unique())  # Inspect unique values


int64
[0 1 2 3 4 5 6 7 8 9]


In [570]:
dru.sim_init_year

2015

In [571]:
?dru.clean_ssp_out_data

Signature: dru.clean_ssp_out_data(ssp_out_df)
Docstring:
Cleans the SSP output data by adding a year column and filtering for the comparison year.
Args:
    ssp_out_df (pd.DataFrame): The SSP output data frame to be cleaned.
Returns:
    pd.DataFrame: The cleaned SSP output data frame containing only the data for the comparison year.
Raises:
    ValueError: If the comparison year is not present in the simulation data.
File:      ~/decision_sciences/ssp_data_calibration/src/utilities/diff_reports.py
Type:      method

In [572]:
dru.clean_ssp_out_data(ssp_out)

,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,area_agrc_crops_other_woody_perennial,area_agrc_crops_pulses,...,yield_agrc_herbs_and_other_perennial_crops_tonne,yield_agrc_nuts_tonne,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne,year
0,0,128992.475401,286925.570284,59534.988647,155176.845408,200241.524314,109285.29166,144565.284932,96468.731604,66425.612333,...,5.781814e+06,65829.816853,279108.581486,50211.9748,175242.943363,0.0,0.0,2.050396e+06,725849.288265,2015


In [573]:
ssp_emissions_report = dru.generate_ssp_emissions_report(ssp_out)
ssp_emissions_report

,subsector,gas,edgar_class,ssp_emission
0,lvst,ch4,AG - Livestock:CH4,1.938988
1,lsmm,ch4,AG - Livestock:CH4,0.040259
2,lsmm,n2o,AG - Livestock:N2O,0.165996
3,agrc,co2,AG - Crops:CO2,1.316953
4,agrc,ch4,AG - Crops:CH4,0.214505
...,...,...,...,...
63,soil,co2,LULUCF - Organic Soil:CO2,0.110122
64,soil,n2o,LULUCF - Organic Soil:N2O,1.031985
65,ccsq,ch4,CCSQ:CH4,0.000000
66,ccsq,co2,CCSQ:CO2,0.000000


In [574]:
# check energy subsector
energy_subsectors = ['trns', 'entc', 'fgtv', 'scoe', 'inen', 'ccsq']
ssp_emissions_report[ssp_emissions_report.subsector.isin(energy_subsectors)]

,subsector,gas,edgar_class,ssp_emission
6,scoe,co2,EN - Building:CO2,7.494763
7,scoe,ch4,EN - Building:CH4,0.019117
8,scoe,n2o,EN - Building:N2O,0.033724
9,entc,co2,EN - Electricity/Heat:CO2,NaN
10,entc,ch4,EN - Electricity/Heat:CH4,NaN
11,entc,n2o,EN - Electricity/Heat:N2O,NaN
12,fgtv,co2,EN - Fugitive Emissions:CO2,NaN
13,fgtv,ch4,EN - Fugitive Emissions:CH4,NaN
14,fgtv,n2o,EN - Fugitive Emissions:N2O,NaN
15,inen,co2,EN - Manufacturing/Construction:CO2,4.042617


In [575]:
dru.model_failed_flag

False

In [576]:
ssp_emissions_report['ssp_emission'].min()

-1.2470138668192876

In [577]:
ssp_emissions_report.sort_values(by=['subsector', 'gas']).tail(20)

,subsector,gas,edgar_class,ssp_emission
52,ippu,sf6,IN - Industrial Processes:SF6,0.000000
61,lndu,ch4,LULUCF - Deforestation:CH4,0.417046
60,lndu,co2,LULUCF - Deforestation:CO2,0.242741
62,lndu,co2,LULUCF - Other Land:CO2,0.025623
1,lsmm,ch4,AG - Livestock:CH4,0.040259
2,lsmm,n2o,AG - Livestock:N2O,0.165996
0,lvst,ch4,AG - Livestock:CH4,1.938988
7,scoe,ch4,EN - Building:CH4,0.019117
6,scoe,co2,EN - Building:CO2,7.494763
8,scoe,n2o,EN - Building:N2O,0.033724


In [578]:
df_report = dru.merge_ssp_with_edgar(ssp_emissions_report, edgar_emissions)
df_report

,subsector,gas,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,direct_weight,norm_weight,log_weight,diff,squared_diff
0,lvst,ch4,AG - Livestock:CH4,1.938988,HRV,0.841458,2015,0.841459,0.841459,0.025337,0.610558,1.304317,1.204570
1,lsmm,ch4,AG - Livestock:CH4,0.040259,HRV,0.841458,2015,0.841459,0.841459,0.025337,0.610558,-0.952156,0.641921
2,lsmm,n2o,AG - Livestock:N2O,0.165996,HRV,0.110193,2015,0.110194,0.110194,0.003318,0.104534,0.506401,0.003114
3,agrc,co2,AG - Crops:CO2,1.316953,HRV,0.095857,2015,0.095858,0.095858,0.002886,0.091537,12.738568,1.491074
4,agrc,ch4,AG - Crops:CH4,0.214505,HRV,0.000227,2015,0.000228,0.000228,0.000007,0.000227,941.296070,0.045915
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,soil,co2,LULUCF - Organic Soil:CO2,0.110122,HRV,0.092259,2015,0.092260,0.092260,0.002778,0.088248,0.193601,0.000319
64,soil,n2o,LULUCF - Organic Soil:N2O,1.031985,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN
65,ccsq,ch4,CCSQ:CH4,0.000000,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN
66,ccsq,co2,CCSQ:CO2,0.000000,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN


In [579]:
# print the df with -999 values anywhere
df_report[df_report['edgar_emission'] == -999]

,subsector,gas,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,direct_weight,norm_weight,log_weight,diff,squared_diff


In [580]:
# get duplicated edgar_class
df_report[df_report.duplicated(subset=['edgar_class'], keep=False)]

,subsector,gas,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,direct_weight,norm_weight,log_weight,diff,squared_diff
0,lvst,ch4,AG - Livestock:CH4,1.938988e+00,HRV,0.841458,2015,0.841459,0.841459,0.025337,0.610558,1.304317,1.204570
1,lsmm,ch4,AG - Livestock:CH4,4.025905e-02,HRV,0.841458,2015,0.841459,0.841459,0.025337,0.610558,-0.952156,0.641921
24,ippu,c2f6,IN - Industrial Processes:HFC,0.000000e+00,HRV,0.019043,2015,0.019044,0.019044,0.000573,0.018864,-1.000000,0.000363
25,ippu,c2h3f3,IN - Industrial Processes:HFC,2.203381e-01,HRV,0.019043,2015,0.019044,0.019044,0.000573,0.018864,10.570216,0.040520
26,ippu,c2hf5,IN - Industrial Processes:HFC,2.003794e-01,HRV,0.019043,2015,0.019044,0.019044,0.000573,0.018864,9.522160,0.032883
27,ippu,c3f8,IN - Industrial Processes:HFC,0.000000e+00,HRV,0.019043,2015,0.019044,0.019044,0.000573,0.018864,-1.000000,0.000363
28,ippu,c3h2f6,IN - Industrial Processes:HFC,0.000000e+00,HRV,0.019043,2015,0.019044,0.019044,0.000573,0.018864,-1.000000,0.000363
29,ippu,c3h3f5,IN - Industrial Processes:HFC,0.000000e+00,HRV,0.019043,2015,0.019044,0.019044,0.000573,0.018864,-1.000000,0.000363
30,ippu,c3hf7,IN - Industrial Processes:HFC,0.000000e+00,HRV,0.019043,2015,0.019044,0.019044,0.000573,0.018864,-1.000000,0.000363
31,ippu,c4f10,IN - Industrial Processes:HFC,0.000000e+00,HRV,0.019043,2015,0.019044,0.019044,0.000573,0.018864,-1.000000,0.000363


In [581]:
df_report['norm_weight'].sum()

0.9999999999999999

In [582]:
dru.run_report_generator(edgar_emissions, ssp_out)

In [583]:
dru.sectoral_emission_report[dru.sectoral_emission_report['subsector'].isin(energy_subsectors)]

,subsector,gas,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,direct_weight,norm_weight,log_weight,diff,squared_diff
6,scoe,co2,EN - Building:CO2,7.494763,HRV,2.726867,2015,2.726868,2.726868,0.082110,1.315568,1.748488,2.273282e+01
7,scoe,ch4,EN - Building:CH4,0.019117,HRV,0.416708,2015,0.416709,0.416709,0.012548,0.348336,-0.954123,1.580794e-01
8,scoe,n2o,EN - Building:N2O,0.033724,HRV,0.104122,2015,0.104123,0.104123,0.003135,0.099050,-0.676113,4.955972e-03
9,entc,co2,EN - Electricity/Heat:CO2,NaN,HRV,3.297328,2015,3.297329,3.297329,0.099287,1.457993,NaN,NaN
10,entc,ch4,EN - Electricity/Heat:CH4,NaN,HRV,0.004396,2015,0.004397,0.004397,0.000132,0.004386,NaN,NaN
11,entc,n2o,EN - Electricity/Heat:N2O,NaN,HRV,0.011222,2015,0.011223,0.011223,0.000338,0.011160,NaN,NaN
12,fgtv,co2,EN - Fugitive Emissions:CO2,NaN,HRV,1.392715,2015,1.392716,1.392716,0.041937,0.872429,NaN,NaN
13,fgtv,ch4,EN - Fugitive Emissions:CH4,NaN,HRV,1.046727,2015,1.046728,1.046728,0.031518,0.716242,NaN,NaN
14,fgtv,n2o,EN - Fugitive Emissions:N2O,NaN,HRV,0.001680,2015,0.001681,0.001681,0.000051,0.001679,NaN,NaN
15,inen,co2,EN - Manufacturing/Construction:CO2,4.042617,HRV,2.148484,2015,2.148485,2.148485,0.064694,1.146921,0.881612,3.587734e+00


In [584]:
dru.sectoral_emission_report.tail(10)

,subsector,gas,edgar_class,ssp_emission,iso_alpha_3,edgar_emission,year,edgar_emission_epsilon,direct_weight,norm_weight,log_weight,diff,squared_diff
58,frst,co2,LULUCF - Forest Land:CO2,-1.247014,HRV,-6.547104,2015,-6.547103,6.547105,0.197142,2.021164,-0.809532,28.090943
59,frst,ch4,LULUCF - Forest Land:CH4,0.126491,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN
60,lndu,co2,LULUCF - Deforestation:CO2,0.242741,HRV,0.060840,2015,0.060841,0.060841,0.001832,0.059061,2.989786,0.033088
61,lndu,ch4,LULUCF - Deforestation:CH4,0.417046,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN
62,lndu,co2,LULUCF - Other Land:CO2,0.025623,HRV,0.820237,2015,0.820238,0.820238,0.024698,0.598967,-0.968761,0.631413
63,soil,co2,LULUCF - Organic Soil:CO2,0.110122,HRV,0.092259,2015,0.092260,0.092260,0.002778,0.088248,0.193601,0.000319
64,soil,n2o,LULUCF - Organic Soil:N2O,1.031985,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN
65,ccsq,ch4,CCSQ:CH4,0.000000,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN
66,ccsq,co2,CCSQ:CO2,0.000000,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN
67,ccsq,n2o,CCSQ:N2O,0.000000,NaN,NaN,2015,NaN,NaN,NaN,NaN,NaN,NaN


In [585]:
dru.subsector_emission_report

,subsector,ssp_emission,edgar_emission_epsilon,diff,squared_diff,year
0,agrc,1.778561,1.134142,0.568199,0.415276,2015
1,ccsq,NaN,0.000000,NaN,NaN,2015
2,entc,NaN,3.312949,NaN,NaN,2015
3,fgtv,NaN,2.441124,NaN,NaN,2015
4,frst,-1.120523,-6.547103,-0.828852,29.447773,2015
5,inen,4.069753,2.157343,0.886465,3.657311,2015
6,ippu,11.167588,3.883249,1.875836,53.061584,2015
7,lndu,0.685409,0.881078,-0.222079,0.038286,2015
8,lsmm,0.206255,0.951652,-0.783267,0.555618,2015
9,lvst,1.938988,0.841459,1.304317,1.204570,2015
